In [3]:
import torch
import pytorch_lightning as pl
import torchvision
from torchvision.datasets import CIFAR10
from matplotlib import pyplot as plt
import numpy as np
from torch.nn import functional as F
from torch import nn



In [4]:
print(torch.cuda.is_available())

True


In [20]:
class CIFARCLassifier(pl.LightningModule):
    def __init__(self):
        super(CIFARCLassifier, self).__init__()

        self.conv1 = nn.Conv2d(3, 32, 5, padding=2)
        self.bn1 = nn.BatchNorm2d(32)
        self.pool = nn.MaxPool2d(3, stride=2, padding=1)
        self.conv2 = nn.Conv2d(32, 16, 5, padding=2)
        self.bn2 = nn.BatchNorm2d(16)
        self.conv3 = nn.Conv2d(16, 32, 5, padding=2)
        self.bn3 = nn.BatchNorm2d(32)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(32 * 4 * 4, 128)
        self.do1 = nn.Dropout(0.25)
        self.fc2 = nn.Linear(128, 10)

    
    def forward(self, x):
        x = self.pool(self.bn1(F.relu(self.conv1(x))))
        x = self.pool(self.bn2(F.relu(self.conv2(x))))
        x = self.pool(self.bn3(F.relu(self.conv3(x))))
        x = self.flatten(x)
        x = self.do1(F.relu(self.fc1(x)))
        x = F.relu(self.fc2(x))
        return F.log_softmax(x, dim=1)

    def prepare_data(self):
        #download the data, augment and normalize them
        self.cifar_train = CIFAR10(root="data/", train=True, transform=torchvision.transforms.Compose([torchvision.transforms.RandomHorizontalFlip(), torchvision.transforms.ToTensor(), torchvision.transforms.Normalize(0.5, 0.5)]))

        self.cifar_val = CIFAR10(root="data/", train=False, transform=torchvision.transforms.Compose([ torchvision.transforms.ToTensor(), torchvision.transforms.Normalize(0.5, 0.5)]))

    def train_dataloader(self):
        return torch.utils.data.DataLoader(self.cifar_train, batch_size=128, shuffle=True)

    def val_dataloader(self):
        return torch.utils.data.DataLoader(self.cifar_val, batch_size=128, shuffle=False)


    def training_step(self, train_batch, batch_idx):
        #1 batch of training
        data, label = train_batch
        out = self.forward(data)
        loss = F.nll_loss(out, label)
        result = pl.TrainResult(loss)
        prediction = out.argmax(dim=1, keepdim=True).squeeze()

        correct =  prediction.eq(label.view_as(prediction)).sum().item()
        return {'loss' : loss, 'correct' : correct}
    
    def training_epoch_end(self, outputs):
        #end of an epoch
        #calculate the average loss of this epoch
        avg_loss = torch.stack([x['loss'] for x in outputs]).mean()
        accuracy = 100 * (sum([x['correct'] for x in outputs]) / float(len(self.cifar_train)))


        logs = {'train_loss': avg_loss, 'train_accuracy': accuracy}
        return {'loss' : avg_loss, 'log' : logs}

    def validation_step(self, validation_batch, batch_idx):
        data, label = validation_batch
        out = self.forward(data)
        loss = F.nll_loss(out, label)
        prediction = out.argmax(dim=1, keepdim=True).squeeze()

        correct =  prediction.eq(label.view_as(prediction)).sum().item()
        return {'loss' : loss, 'correct' : correct}

    def validation_epoch_end(self, outputs):
        #end of an epoch
        #calculate the average loss of this epoch
        avg_loss = torch.stack([x['loss'] for x in outputs]).mean()
        accuracy = 100 * (sum([x['correct'] for x in outputs]) / float(len(self.cifar_val)))

        logs = {'validation_loss': avg_loss, 'validation_accuracy': accuracy}
        return {'loss' : avg_loss, 'log' : logs}
    
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.001)





In [21]:
from pytorch_lightning.loggers import TensorBoardLogger

logger = TensorBoardLogger('tb_logs', name="cifar10_documentation_model")

model = CIFARCLassifier()
model.prepare_data()
model.train_dataloader()
trainer = pl.Trainer(max_epochs=10, logger=logger, gpus=[0], fast_dev_run=False)

trainer.fit(model)


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]

   | Name    | Type        | Params
-----------------------------------------
0  | conv1   | Conv2d      | 2 K   
1  | bn1     | BatchNorm2d | 64    
2  | pool    | MaxPool2d   | 0     
3  | conv2   | Conv2d      | 12 K  
4  | bn2     | BatchNorm2d | 32    
5  | conv3   | Conv2d      | 12 K  
6  | bn3     | BatchNorm2d | 64    
7  | flatten | Flatten     | 0     
8  | fc1     | Linear      | 65 K  
9  | do1     | Dropout     | 0     
10 | fc2     | Linear      | 1 K   


Saving latest checkpoint..



1